In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import cufflinks as cf
import envelope_plot as ep
from ipywidgets import interact
import ipywidgets as widgets
import os

cf.go_offline()
plotly.offline.init_notebook_mode(connected=True)

def extract_keys(my_dict, key_list):
    return {k: my_dict[k] for k in key_list}
    

# Prepare data

Every domain instance requires the optimal path length which is
read in via a json results file in the same format as the experiment results.

Specify the paths to the results yielding information on the optimal plans below.
Then specify paths to the experiment results file. A dataframe will be constructed
using this information


In [ ]:
optimal_plans = ['../results/astar_minima1500.json','../results/astar_orz100d.json',
                 '../results/astar_korf4real.json']
experiments = ['../results/results_bi.json', '../results/results_tba.json']

experiment_data = ep.analyze_results(optimal_plans, experiments)

# Plot data

Please use the drop down menus to select the domain instance and the plot type

In [ ]:
domains = {'Minima 1500x1500':'minima1500', 'Dragon Age: Origins':'orz100d', 'Sliding Tile - Korf 4x4','tiles.korf/4/real'}
# domains = {'Dragon Age: Origins':'orz100d'}
master_config = {'weight': 'W', 'lookaheadStrategy': 'Look', 'envelopeSearchStrategy': 'Env-L'}

just_weight = extract_keys(master_config, ['weight'])
wLookaheadConfig = extract_keys(master_config, ['weight', 'lookaheadStrategy'])
biEsConfig = extract_keys(master_config, ['weight', 'lookaheadStrategy', 'envelopeSearchStrategy'])

within_opt_data = ep.prepare_for_within_opt_plot(experiment_data,domains.values(),{
    'LSS_LRTA_STAR':{},
    'BI_ES': biEsConfig,
    'BACK_ES': just_weight,
    'TIME_BOUNDED_A_STAR': wLookaheadConfig
})

@interact(
    domain=domains,
    display_type=['Plot','Table'],
    algorithms=widgets.SelectMultiple(
        options={
            'TBA*':'TIME_BOUNDED_A_STAR',
            'LSS-LRTA*':'LSS_LRTA_STAR',
            'Bidirectional Envelope': 'BI_ES',
            'Backward Envelope': 'BACK_ES'
        },
        description='Algorithms (Select multiple)',
        value=[]
    ),
    weights=widgets.SelectMultiple(
        options=list(filter(lambda x: x==x, experiment_data.weight.unique())), # remove NaN
        description='Weights (Select multiple, or none for all)',
        value=[]
    )
)
def plot_within_opt(domain, display_type, algorithms, weights):
    data_copy = within_opt_data[domain].copy() # for safety - don't modify original
    
    if len(algorithms) == 0:
        return
    
    trimmed_df = pd.DataFrame({}, columns=data_copy.columns)
    
    for alg in algorithms:
        alg_data = data_copy[data_copy.algCode == alg]
        # filter by weight if weight is used as a parameter
        if len(weights) > 0:
            alg_data = alg_data[alg_data.weight.isnull() | alg_data.weight.isin(weights)]
        
        trimmed_df = trimmed_df.append(alg_data)
        
    if display_type == 'Table':
        table = ff.create_table(trimmed_df)
        table.layout.update({'title':domain})
        plotly.iplot(table, filename='jupyter-table')
    elif display_type == 'Plot':
        plot = trimmed_df.iplot(asFigure=True, kind='scatter', mode='lines+markers',
                                x='actionDuration', y='withinOpt', logx=True,
                                xTitle='actionDuration', yTitle='withinOpt',
                                categories='algorithmName',
                                colorscale='spectral', title=domain)
        plotly.iplot(plot, filename='jupyter-plot')
        

# table1 = ff.create_table(within_opt_data['minima1500'])
# plotly.iplot(table1, filename='jupyter-test1')

# table2 = ff.create_table(within_opt_data['orz100d'])
# plotly.iplot(table2, filename='jupyter-test2')